In [2]:
# Imports Libs for:

# Web scraping
import requests
from bs4 import BeautifulSoup
from myfuncs import * # Self-defined functions for pulling data from specific sites
from dotenv import load_dotenv
import os

# Data analysis
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import MDS
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score #, calinski_harabasz_score

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import geopandas as gpd

# Set random seed
np.random.seed(42)


/Users/ningnong/Desktop/MADS/SIADS696_MilestoneII/Air Travel Project/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


# Data Extraction & Preprocessing

## State-level Data

### FRED API

In [ ]:
url = 'https://fred.stlouisfed.org/release/tables?rid=249&eid=259462' # Median household income by state
variable = 'Median Household Income'

income_df = extract_FRED_data(url,variable)
income_df.head()

In [ ]:
url = 'https://fred.stlouisfed.org/release/tables?eid=840687&rid=116'
variable = 'Unemployment Rate'
UE_df = extract_FRED_data(url,variable)
UE_df.head()

In [ ]:
url = 'https://fred.stlouisfed.org/release/tables?eid=259194&rid=118'
variable = 'Population'
pop_df = extract_FRED_data(url,variable)
pop_df.head()

In [ ]:
# Compile FRED data: income_df, UE_df, pop_df

fred_df = pd.merge(income_df, UE_df, on=['State Name', 'Year'], how='inner') \
           .merge(pop_df, on=['State Name', 'Year'], how='inner')
# fred_df.to_csv('assets/FRED_data.csv', index=None)
# fred_df = pd.read_csv('assets/FRED_data.csv')
fred_df.head()

### US Census Bureau: American Community Survey (ACS)

In [ ]:
api_key_USCB = os.getenv('API_KEY_USCB')

acs_2023_df = extract_and_preprocess_ACS_data(api_key=api_key_USCB, year=2023, state_code=None)
acs_2022_df =  extract_and_preprocess_ACS_data(api_key=api_key_USCB, year=2022, state_code=None)

In [ ]:
acs_df = pd.concat([acs_2023_df,acs_2022_df],ignore_index=True)
acs_df.head()

In [ ]:
acs_df.Year.unique()

### State Information

In [ ]:
# Map state FIPS code to state names

state_df = extract_state_mapper()
state_df.to_csv('assets/state_info.csv', index=None)
# state_df = pd.read_csv('assets/state_info.csv')
state_df.head()

In [ ]:
# Merge state name to acs_df based on FIPS code
acs_df= pd.merge(acs_df, state_df, on = 'State Code (FIPS)', how = 'left').dropna()
acs_df.to_csv('assets/acs_df.csv',index=None)
# acs_df = pd.read_csv('assets/acs_df.csv')
acs_df.head()

### T100 Flight Data (2023) 
- for Clustering of states

In [ ]:
# Domestic flight by origin airports (2023)
# Data: Flight volume by Airport Code; this will be our main dataset
# Extraction: csv download from https://equity-data.dot.gov/datasets/17e9a793c7cf47c8b64dab92da55dfe5/about

fp_flights = 'assets/T100_Domestic_Market_and_Segment_Data_-3591723781169319541.csv'
flight_df = pd.read_csv(fp_flights)
flight_df.columns = flight_df.columns.str.title()
flight_df.rename(columns={'Origin': 'Airport Code'}, inplace=True)
flight_df.head() # Here, origin airport names ('origin') are abbreviated by the respective Airport Codes.

In [112]:
# Map airports to their respective states based on Airport Code
df_airports = extract_airport_info()
df_airports.to_csv('assets/airport_data.csv',index=False)
df_airports.head()

,Airport Code,Airport Name,City,State Code (USPS)
0,ABE,Lehigh Valley International,Allentown,PA
1,ABI,Abilene Regional,Abilene,TX
2,ABQ,Albuquerque International,Albuquerque,NM
3,ACK,Nantucket Memorial,Nantucket,MA
4,ADQ,Kodiak,Kodiak,AK


In [ ]:
# df_airports['Airport Code'].unique() # ok

# Edit done for the function extract_airport_info()
# # df_airports['State Code (USPS)'].unique() # contain trailing whitespaces
# df_airports['State Code (USPS)'] = df_airports['State Code (USPS)'].str.strip()
# df_airports['State Code (USPS)'].unique() 

In [ ]:
# Add state info (State Code (USPS)) to flight_df
flight_df = pd.merge(flight_df,df_airports, on='Airport Code', how='left').dropna()
flight_df.head()

In [ ]:
flight_df[flight_df.isna().any(axis=1)] 

In [ ]:
# Add complete state-level info to flight_df 
flight_df = pd.merge(flight_df, state_df, on="State Code (USPS)", how='left')
flight_df.head()

In [ ]:
flight_df[flight_df.isna().any(axis=1)] #some rows failed to merge based on state code

In [ ]:
flight_na = flight_df[flight_df.isna().any(axis=1)] 
flight_na['State Code (USPS)'].unique()

In [ ]:
hawaii_state_info = state_df[state_df['State Code (USPS)'] == 'HI']
hawaii_FIPS = hawaii_state_info['State Code (FIPS)'].values[0]
hawaii_USPS = hawaii_state_info['State Code (FIPS)'].values[0]
hawaii_state_name = hawaii_state_info['State Code (FIPS)'].values[0]

hawaii_state_info

In [ ]:
# Only Hawaii is a state. Mariana Islands, Puerto Rico (PR) and the U.S. Virgin Islands (VI) are not considered states in the United States
# Perform correction for Hawaii
flight_df.loc[flight_df['State Code (USPS)'].str.contains('HI'), ['State Code (USPS)', 'State Code (FIPS)', 'State Name']] = ['HI', 15, 'Hawaii']
flight_df[flight_df['State Code (USPS)'] == 'Hawaii, HI']

In [ ]:
flight_df[flight_df['State Code (USPS)'] == 'HI']

In [ ]:
flight_df[flight_df.isna().any(axis=1)] 

In [ ]:
# These are non-state rows (PR, VI, Mariana Islands), let's drop them
flight_df.dropna(inplace=True)
# flight_df[flight_df.isna().any(axis=1)]  # No more NA rowsß
# flight_df.to_csv('assets/flight_data.csv',index=None)
# flight_df = pd.read_csv('assets/flight_data.csv')
flight_df.head()

In [ ]:
flight_df.columns

## Airport/Airline-level Data

### BTS Flight data (2017-2024)

In [20]:
airports_airlines = os.listdir('assets/bts_data')
airports_airlines[:5]

['PHL_Delta.csv',
 'ORD_Frontier.csv',
 'CLE_Delta.csv',
 'ATL_American.csv',
 'IAD_United.csv']

In [28]:
i=0
fpath = f'assets/bts_data/%s'%(airports_airlines[i])
with open(fpath, 'r') as file:
    print(file.read())

Detailed Statistics Departures  
Origin Airport: Philadelphia, PA: Philadelphia International (PHL)
Airline: Delta Airlines Inc. (DL)
Month(s): January, February, March, April, May, June, July, August, September, October, November, and December
Day(s): 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30 and 31
Year(s): 2017, 2018, 2019, 2021, 2022, 2023 and 2024

Carrier Code,Date (MM/DD/YYYY),Flight Number,Tail Number,Destination Airport,Actual departure time
"DL","01/01/2017","1356","N706TW","SLC","16:24"
"DL","01/01/2017","1494","N923AT","ATL","11:23"
"DL","01/01/2017","1561","N341NB","ATL","14:12"
"DL","01/01/2017","1661","N958AT","ATL","18:09"
"DL","01/01/2017","2061","N964DN","ATL","07:27"
"DL","01/01/2017","2261","N925DN","ATL","17:13"
"DL","01/01/2017","2505","N341NB","ATL","06:10"
"DL","01/01/2017","2568","N938DL","ATL","08:35"
"DL","01/01/2018","0805","N983DL","ATL","08:20"
"DL","01/01/2018","1241","N904DL","ATL","16:1

In [69]:
airports = ['PHL','CLE','ATL','ORD','EWR','IAD','LAX','DFW','MIA']
airlines = ['Delta','American','United','Southwest','Spirit','Frontier']
fpath = 'assets/bts_data'
airport_dfs = []
for airport in airports:
    airport_airlines = []
    for airline in airlines:
        fname = f'{airport}_{airline}.csv'
        if fname in os.listdir('assets/bts_data'):
            airport_airline = pd.read_csv(f'{fpath}/{airport}_{airline}.csv',skiprows=np.arange(0,7), delimiter=',')
            airport_airline.rename(columns={
                "Carrier Code":"Airline Code", 
                "Destination Airport": "Destination Airport Code",
                },
                inplace=True)
            airport_airline.drop(columns=['Actual departure time','Tail Number','Flight Number'],inplace=True)
            airport_airline['Airline'] = airline
            airport_airline.dropna(inplace=True)
            airport_airlines.append(airport_airline.reset_index(drop=True))
        else: 
            continue
    airport_df = pd.concat(airport_airlines)
    airport_df['Origin Airport Code'] = airport
    airport_dfs.append(airport_df.reset_index(drop=True))
flight_df2 = pd.concat(airport_dfs)

del airport_df, airport_dfs, airport_airline, airport_airlines

flight_df2.rename(columns={'Date (MM/DD/YYYY)': 'Date'}, inplace=True)
flight_df2['Date'] = pd.to_datetime(flight_df2['Date'], format='%m/%d/%Y')
flight_df2['Year'] = flight_df2['Date'].dt.year
flight_df2['Month'] = flight_df2['Date'].dt.month
flight_df2['Day'] = flight_df2['Date'].dt.day

flight_df2

,Airline Code,Date,Destination Airport Code,Airline,Origin Airport Code,Year,Month,Day
0,DL,2017-01-01,SLC,Delta,PHL,2017,1,1
1,DL,2017-01-01,ATL,Delta,PHL,2017,1,1
2,DL,2017-01-01,ATL,Delta,PHL,2017,1,1
3,DL,2017-01-01,ATL,Delta,PHL,2017,1,1
4,DL,2017-01-01,ATL,Delta,PHL,2017,1,1
...,...,...,...,...,...,...,...,...
476569,F9,2023-12-31,CLE,Frontier,MIA,2023,12,31
476570,F9,2023-12-31,MDW,Frontier,MIA,2023,12,31
476571,F9,2023-12-31,ATL,Frontier,MIA,2023,12,31
476572,F9,2023-12-31,PHL,Frontier,MIA,2023,12,31


In [70]:
flight_df2.to_csv('assets/flight_data2.csv', index=False)

# EDA

In [ ]:
# Filter state-level socio-demographic features based on the presence/absence of correlation with enplanements. (Qn: Which sociodemographic variables should we include for clustering of states? )

# First, calculate state statistics for Enplanement (air travel volume) in flight_df
flight_byState = flight_df[['State Name','Enplanements']].groupby('State Name').mean().reset_index()

# Create a df to explore the relationships for 2023 data
df = pd.merge(flight_byState, acs_df.query('Year==2023'), on='State Name', how='left')
df.head()

In [ ]:
df.select_dtypes('number').columns

In [ ]:
sociodemo_cols = [
    'Median Household Income', 
    'Per Capita Income',
    'Gini Index of Income Inequality',
    'Total Population', 
    'Median Age',
    'Median Home Value', 
    'Unemployment Rate',
    'Percent Foreigners'
    ]

plt.figure(figsize=(5*len(sociodemo_cols), 5))  
for i in range(len(sociodemo_cols)):
    plt.subplot(1,len(sociodemo_cols),i+1)
    plt.scatter(df['Enplanements'],df[sociodemo_cols[i]])
    plt.xlabel(sociodemo_cols[i])
    if i==0:
        plt.ylabel('Enplanements')
    plt.suptitle('Enplanements vs. Socio-demographic features by State', fontsize=20, weight='bold')
plt.savefig('visualizations/enplanements_vs_sociodemographics.png', dpi=300)
plt.show()

- Mainly positive, linear correlations between Enplanements and the selected sociodemographic variables except Median age. --> let's drop median age col from subsequent clustering.
- Most sociodemographic variables appear to be right-skewed

In [ ]:
df.drop(columns=['Median Age'],inplace=True)

In [ ]:
acs_df.Year.unique()
acs_df.columns

In [ ]:
# Create a SPLOM to see the relationships/correlations among Socio-demographic features
# SPLOM for 2022 and 2023 data (SPLOM by year)
img_name = 'acs_SPLOMs_byYear'

tmp = acs_df[[col for col in acs_df.columns if 'State' not in col]].copy()
sns.pairplot(tmp, hue="Year", palette='deep')
plt.savefig(f'visualizations/{img_name}.png', dpi=300)
plt.show()

- Similar distributions for 2022 and 2023. Let's use 2023 for analysis 

In [ ]:
# SPLOM for 2023 data
sns.pairplot(df[[col for col in df.columns if col not in ['Enplanements','Year','State Code (FIPS)']]])
# plt.savefig('visualizations/SPLOM_sociodemographics.png',dpi=300)
plt.show()

# Unsupervised Learning

## Dimensionality Reduction

### PCA

In [ ]:
df.head()

In [ ]:
df.select_dtypes('number').columns

In [ ]:
# Normalize for PCA
scaler = StandardScaler()
df_scaled = df.copy() # using df with 2023 data

# Select numeric cols
numeric_cols = [col for col in df.select_dtypes('number').columns if (col != "Year") & (col != "Enplanements")]
df_scaled[numeric_cols] = scaler.fit_transform(df_scaled[numeric_cols])

# Perform PCA 
pca = PCA() # Default n_components = number of existing features
pca.fit(df_scaled[numeric_cols])


# Select n_components:

# Option 1 (not chosen): Based on Explained Variance -> 6
explainedVariance = pca.explained_variance_ratio_
explainedVariance_cum = np.cumsum(pca.explained_variance_ratio_)
explainedVariance_cum
# n_components = np.argmax(explainedVariance_cum >= 0.95) + 1 #np.argmax returns the index of the first occurence of max value ('1') - i.e. index of first element which satisfy the criteria >0.95
# n_components #np.int64(6)

# Option 2 (chosen): Based on Visual inspection of Scree plot using Elbow method -> 3
img_name = 'PCA_ExplainedVariance_ScreePlot'
plt.plot(np.arange(1, len(explainedVariance) + 1), explainedVariance, marker='o')
plt.xlabel('Number of Components')
plt.ylabel('Explained Variance Ratio')
plt.title('Scree Plot')

plt.savefig(f'visualizations/{img_name}.png', dpi=300)
plt.show() # choose n=3

- Elbow observed at n=3

In [ ]:
# Final Scree Plot

# Option 2 (chosen): Based on visual inspection -> 3 components
n_components = 3
explained_variance_at_3 = explainedVariance_cum[n_components - 1]  # Cumulative explained variance at PC = 3

# Plot Scree Plot with vertical line at n = 3
plt.plot(np.arange(1, len(explainedVariance) + 1), explainedVariance, marker='o', label='Explained Variance Ratio')

# Add vertical line at PC = 3
plt.axvline(x=n_components, color='g', linestyle='--', label=f'n={n_components}')

# Annotate with explained variance at PC = 3
plt.text(n_components + 0.1, explainedVariance[0], f'Total Explained Variance = {explained_variance_at_3:.2f}', color='green')

# Labels and title
plt.xlabel('Number of Components')
plt.ylabel('Explained Variance Ratio')
plt.title('Scree Plot')

# Save and show plot
img_name = 'PCA_ExplainedVariance_ScreePlot'
plt.legend()
plt.savefig(f'visualizations/{img_name}.png', dpi=300)
plt.show()

In [ ]:
# Perform final PCA with 3 components
n=3
pca = PCA(n_components=n)
pca_result = pca.fit_transform(df_scaled[numeric_cols])

# PCA results 
pca_df = pd.DataFrame(pca_result, columns=[f'PC{i+1}' for i in range(n)])
# Concatenate the original data with PCA components
pca_df = pd.concat([df.reset_index(drop=True), pca_df], axis=1)
pca_df.to_csv('assets/pca_df.csv', index=None) 
pca_df.head()

In [ ]:
# pca_df.head()
print(pca_df.isnull().sum())


In [ ]:
# Look into how the original features contribute to each PC's

# Component contributions (loadings)
component_cont = pca.components_
# component_cont
pca_components_df = pd.DataFrame(component_cont, columns=numeric_cols, index=[f'PC{i+1}' for i in range(n)])
pca_components_df.head()

In [ ]:
pca_components_df.to_csv('assets/PCA_components.csv', index=False)

In [ ]:
# Create a heatmap to visualize the component contributions (loadings)
plt.figure(figsize=(10, 8))
sns.heatmap(pca_components_df, annot=True, cmap='coolwarm', center=0, linewidths=0.5, fmt=".2f")

# Set plot labels and title
plt.title('Heatmap of Feature Contributions to Principal Components')
plt.xlabel('Original Features')
plt.ylabel('Principal Components')

# Show the heatmap
plt.tight_layout()
plt.savefig('visualizations/pca_component_contribution_heatmap.png',dpi=300)
plt.show()

**Main PC Components** 
- PC1: Percent Foreigners, Income (Household & Per Capita), Home Value
- PC2: Gini Index, Total Population, Household Income
- PC3: Total Population, Gini Index

In [ ]:
# Analyse PC1 on the original df with both 2022 and 2023 data

# Explained Variance by each component
explainedVariance_byComponent = pca.explained_variance_ratio_

explainedVariance_PC1 = explainedVariance_byComponent[0]
print(f"PC1 explains {explainedVariance_PC1*100:.2f}% of the total variance")

# Create a SPLOM for Main components of PC1 identified earlier
main_cols_pc1 = ['Percent Foreigners','Median Household Income', 'Per Capita Income', 'Median Home Value']
img_name_pc1 = 'acs_PC1_SPLOMs'
cols = main_cols_pc1
img_name = img_name_pc1

tmp = acs_df[cols+['Year']].copy()
pairplot = sns.pairplot(tmp, hue="Year", palette='deep')
pairplot.figure.suptitle('PC1 Component Analysis', y=1.02) 
plt.savefig(f'visualizations/{img_name}.png', dpi=300)
plt.show()

**Observations for PC1**
- Main components of PC1 are linearly correlated. PC1 here aligns with the directions where the most correlated features exist—because these correlations reflect the shared structure in the data.
- The first principal component (PC1) captures the largest proportion of variance. If features are correlated, they will share a significant portion of this variance, and PC1 will represent this shared variance.
- Similar trends for both 2022 and 2023.

In [ ]:
# Analyse PC2
# Main features for PC2
main_cols_pc2 = ['Gini Index of Income Inequality','Total Population','Median Household Income']
img_name_pc2 = 'acs_PC2_SPLOMs'
cols = main_cols_pc2
img_name = img_name_pc2

tmp = acs_df[cols+['Year']].copy()
pairplot = sns.pairplot(tmp, hue="Year", palette='deep')
pairplot.figure.suptitle('PC2 Component Analysis', y=1.02) 
plt.savefig(f'visualizations/{img_name}.png', dpi=300)
plt.show()

**Observations for PC2**
- Much weaker correlations than PC1 components
- Similar trends for both 2022 and 2023, although Median Household Income shifts up slightly

### MDS

In [ ]:
X = df_scaled[numeric_cols]

# Normalize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Run MDS for different numbers of components
stress_values = []
n_components_range = np.arange(1, 11)  # Try MDS with 1 to 10 dimensions

for n_components in n_components_range:
    mds = MDS(n_components=n_components, random_state=42)
    mds.fit(X_scaled)
    stress_values.append(mds.stress_)

# Find optimal number of components (elbow point)
optimal_n_components = 3  # Based on visual inspection

# Plot scree-like plot for MDS stress
plt.plot(n_components_range, stress_values, marker='o', label='Stress Values')

# Add vertical line at the chosen n_components
plt.axvline(x=optimal_n_components, color='g', linestyle='--', label=f'n_components={optimal_n_components}')

# Annotate with stress at the chosen n_components
plt.text(optimal_n_components + 0.1, stress_values[optimal_n_components - 1], 
         f'Stress = {stress_values[optimal_n_components - 1]:.2f}', color='green')

# Labels and title
plt.xlabel('Number of Components')
plt.ylabel('Stress')
plt.title('Scree Plot for MDS')

# Save and show plot
img_name = 'MDS_Stress_ScreePlot'
plt.legend()
plt.savefig(f'visualizations/{img_name}.png', dpi=300)
plt.show()


In [ ]:
# Perform MDS with n_components = 3
n=3
mds = MDS(n_components=n, random_state=42)

# MDS results 
mds_result = mds.fit_transform(df_scaled[numeric_cols])
mds_df = pd.DataFrame(mds_result, columns=[f'MDS_D{i+1}' for i in range(n)])
# Concatenate the original data with mds components
mds_df = pd.concat([df.reset_index(drop=True), mds_df], axis=1)
mds_df.to_csv('assets/mds_df.csv', index=None) 
mds_df.head()

## Clustering

### KMeans

In [ ]:
# Perform KMeans Clustering on PCA results (PCA + KMeans)
pca_cols = [col for col in pca_df.columns if 'PC' in col]
mds_cols = [col for col in mds_df.columns if 'MD' in col]

# Find most appropriate number of clusters
# Define range of K values
K = range(1, 11)

# Store inertia values for each K
# Inertia = within-cluster sum of squares (WCSS) 
inertia_pca = []
inertia_mds = []

for k in K:
    kmeans_pca = KMeans(n_clusters=k, random_state=42)
    kmeans_pca.fit(pca_df[pca_cols])
    inertia_pca.append(kmeans_pca.inertia_)

    kmeans_mds = KMeans(n_clusters=k, random_state=42)
    kmeans_mds.fit(mds_df[mds_cols])
    inertia_mds.append(kmeans_mds.inertia_)

# Plot the Elbow Method graph
img_name = 'KMeans_Elbow'
plt.figure(figsize=(12, 6))  # Set the figure size to be more readable

# Plot for KMeans with PCA
plt.subplot(1, 2, 1)  # First subplot
plt.plot(K, inertia_pca, 'bo-', label='Inertia with PCA')
plt.xlabel('Number of clusters (K)')
plt.ylabel('Inertia')
plt.title('KMeans with PCA')

# Plot for KMeans with MDS
plt.subplot(1, 2, 2)  # Second subplot
plt.plot(K, inertia_mds, 'go-', label='Inertia with MDS')
plt.xlabel('Number of clusters (K)')
plt.ylabel('Inertia')
plt.title('KMeans with MDS')

# Save the figure with the correct image name
plt.savefig(f'visualizations/{img_name}.png', dpi=300)
plt.tight_layout() 
plt.show()



- Choose a common K based on Elbow Methdo: K=3

In [ ]:
# Let's compare silhoutte score for PCA+KMeans vs MDS+KMeans

K=3
kmeans_pca = KMeans(n_clusters=K,random_state=42)
# kmeans_pca.fit(pca_df[pca_cols])
# labels_pca = kmeans_pca.labels_
labels_pca = kmeans_pca.fit_predict(pca_df[pca_cols])

kmeans_mds = KMeans(n_clusters=K,random_state=42)
kmeans_mds.fit(mds_df[mds_cols])
labels_mds = kmeans_mds.labels_

# Compute silhouette scores
silhouette_score_pca = silhouette_score(pca_df[pca_cols], labels_pca)
silhouette_score_mds = silhouette_score(mds_df[mds_cols], labels_mds)
print(f'Silhouette Score for PCA + KMeans: {silhouette_score_pca}')
print(f'Silhouette Score for MDS + KMeans: {silhouette_score_mds}')

PCA + KMeans gives better clusters based on silhouette score.

In [ ]:
# Create a final df to store KMean clusters of data transformed various ways, for ease of visualizations
kmeans_df = pca_df.copy()
kmeans_df['MDS_D1'] = mds_df['MDS_D1']
kmeans_df['MDS_D2'] = mds_df['MDS_D2']
kmeans_df['MDS_D3'] = mds_df['MDS_D3']

kmeans_df["KMeans_PCA Clusters"] = labels_pca
kmeans_df["KMeans_MDS Clusters"] = labels_mds
# kmeans_df.to_csv('assets/kmeans_df.csv',index=None)
kmeans_df.head()

In [ ]:
# Define the DataFrame with PCA columns and clusters
tmp = kmeans_df

# Separate plot for PCA
plt.figure(figsize=(6*3,6))  # Set figure size for PCA
plt.suptitle(f'KMeans Clustering of Travelers\' Sociodemographics - PCA transformed \n(Silhouette score: {round(silhouette_score_pca,2)})', weight='bold', fontsize=18, y=1.05)

label_list = ['PC1', 'PC2', 'PC3']

# Scatter plot PC1 vs PC2
plt.subplot(1, 3, 1)  # 1 row, 3 columns, 1st subplot
for cluster in tmp['KMeans_PCA Clusters'].unique():
    cluster_data = tmp[tmp['KMeans_PCA Clusters'] == cluster]
    plt.scatter(cluster_data['PC1'], cluster_data['PC2'], label=f'Cluster {cluster}', s=50)
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.legend()

# Scatter plot PC1 vs PC3
plt.subplot(1, 3, 2)  # 1 row, 3 columns, 2nd subplot
for cluster in tmp['KMeans_PCA Clusters'].unique():
    cluster_data = tmp[tmp['KMeans_PCA Clusters'] == cluster]
    plt.scatter(cluster_data['PC1'], cluster_data['PC3'], label=f'Cluster {cluster}', s=50)
plt.xlabel('PC1')
plt.ylabel('PC3')
plt.legend()

# Scatter plot PC2 vs PC3
plt.subplot(1, 3, 3)  # 1 row, 3 columns, 3rd subplot
for cluster in tmp['KMeans_PCA Clusters'].unique():
    cluster_data = tmp[tmp['KMeans_PCA Clusters'] == cluster]
    plt.scatter(cluster_data['PC2'], cluster_data['PC3'], label=f'Cluster {cluster}', s=50)
plt.xlabel('PC2')
plt.ylabel('PC3')
plt.legend()

# Adjust layout and save the PCA plot
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.savefig('visualizations/KMeans_Clusters_PCA.png')
plt.show()

In [ ]:
# Separate plot for MDS
plt.figure(figsize=(6*3, 6))  # Set figure size for MDS
plt.suptitle(f'KMeans Clustering of Travelers\' Sociodemographics - MDS transformed \n(Silhouette Score: {round(silhouette_score_mds,2)})', weight='bold', fontsize=18, y=1.05)

label_list = ['MDS_D1', 'MDS_D2', 'MDS_D3']

# Scatter plot MDS_D1 vs MDS_D2
plt.subplot(1, 3, 1)  # 1 row, 3 columns, 1st subplot
for cluster in tmp['KMeans_MDS Clusters'].unique():
    cluster_data = tmp[tmp['KMeans_MDS Clusters'] == cluster]
    plt.scatter(cluster_data['MDS_D1'], cluster_data['MDS_D2'], label=f'Cluster {cluster}', s=50)
plt.xlabel('MDS_D1')
plt.ylabel('MDS_D2')
plt.legend()

# Scatter plot MDS_D1 vs MDS_D3
plt.subplot(1, 3, 2)  # 1 row, 3 columns, 2nd subplot
for cluster in tmp['KMeans_MDS Clusters'].unique():
    cluster_data = tmp[tmp['KMeans_MDS Clusters'] == cluster]
    plt.scatter(cluster_data['MDS_D1'], cluster_data['MDS_D3'], label=f'Cluster {cluster}', s=50)
plt.xlabel('MDS_D1')
plt.ylabel('MDS_D3')
plt.legend()

# Scatter plot MDS_D2 vs MDS_D3
plt.subplot(1, 3, 3)  # 1 row, 3 columns, 3rd subplot
for cluster in tmp['KMeans_MDS Clusters'].unique():
    cluster_data = tmp[tmp['KMeans_MDS Clusters'] == cluster]
    plt.scatter(cluster_data['MDS_D2'], cluster_data['MDS_D3'], label=f'Cluster {cluster}', s=50)
plt.xlabel('MDS_D2')
plt.ylabel('MDS_D3')
plt.legend()

# Adjust layout and save the MDS plot
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.savefig('visualizations/KMeans_Clusters_MDS.png')
plt.show()

### DBSCAN

In [ ]:
# pca+DBSCAN
# Set up parameters
eps_values = [0.9, 1, 1.1, 1.2]
min_samples_values = [5, 6, 7, 8]

X = pca_df[pca_cols].copy()

# OPTIMIZATION
best_eps = None
best_min_samples = None
best_noise_points = None
best_score = -1

# Loop over eps and min_samples values to find the best silhouette score
for eps in eps_values:
    for min_samples in min_samples_values:
        dbscan = DBSCAN(eps=eps, min_samples=min_samples)
        labels = dbscan.fit_predict(X)
        
        if len(set(labels)) > 2:  # Avoids the case where there's one cluster and noise
            score = silhouette_score(X, labels)
            noise_points = np.sum(labels == -1)
            total_points = len(labels)
            print(f"eps: {eps}, min_samples: {min_samples}, score: {score}, and {noise_points}/{total_points} noise points")
            
            if score > best_score:
                best_score = score
                best_eps = eps
                best_min_samples = min_samples
                best_noise_points = noise_points

print(f"Best eps: {best_eps}, best min_samples: {best_min_samples} with Silhouette score: {best_score}, and {best_noise_points}/{total_points} noise points")

In [ ]:
# pca+DBSCAN Clustering (assuming best parameters are already found)
dbscan_best = DBSCAN(eps=best_eps, min_samples=best_min_samples)
best_labels = dbscan_best.fit_predict(X)

# Define a DataFrame with PCA components for better alignment with other plots
pca_df_dbscan = pd.DataFrame(X, columns=['PC1', 'PC2', 'PC3'])
pca_df_dbscan['DBSCAN Clusters'] = best_labels

# Calculate silhouette score for DBSCAN
if len(set(best_labels)) > 1:  # Check if there are more than one cluster
    silhouette_score_dbscan = silhouette_score(X, best_labels)
else:
    silhouette_score_dbscan = -1  # Not applicable if there is only one cluster

# Separate plot for DBSCAN + PCA
plt.figure(figsize=(6*3, 6))  # Set figure size
plt.suptitle(f'DBSCAN Clustering of PCA-transformed Data \n(Silhouette score: {round(silhouette_score_dbscan, 2)})', weight='bold', fontsize=18, y=1.05)

label_list = ['PC1', 'PC2', 'PC3']

# Scatter plot PC1 vs PC2
plt.subplot(1, 3, 1)  # 1 row, 3 columns, 1st subplot
for cluster in pca_df_dbscan['DBSCAN Clusters'].unique():
    cluster_data = pca_df_dbscan[pca_df_dbscan['DBSCAN Clusters'] == cluster]
    label_name = f'Cluster {cluster}' if cluster != -1 else 'Noise'
    plt.scatter(cluster_data['PC1'], cluster_data['PC2'], label=label_name, s=50)
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.legend()

# Scatter plot PC1 vs PC3
plt.subplot(1, 3, 2)  # 1 row, 3 columns, 2nd subplot
for cluster in pca_df_dbscan['DBSCAN Clusters'].unique():
    cluster_data = pca_df_dbscan[pca_df_dbscan['DBSCAN Clusters'] == cluster]
    label_name = f'Cluster {cluster}' if cluster != -1 else 'Noise'
    plt.scatter(cluster_data['PC1'], cluster_data['PC3'], label=label_name, s=50)
plt.xlabel('PC1')
plt.ylabel('PC3')
plt.legend()

# Scatter plot PC2 vs PC3
plt.subplot(1, 3, 3)  # 1 row, 3 columns, 3rd subplot
for cluster in pca_df_dbscan['DBSCAN Clusters'].unique():
    cluster_data = pca_df_dbscan[pca_df_dbscan['DBSCAN Clusters'] == cluster]
    label_name = f'Cluster {cluster}' if cluster != -1 else 'Noise'
    plt.scatter(cluster_data['PC2'], cluster_data['PC3'], label=label_name, s=50)
plt.xlabel('PC2')
plt.ylabel('PC3')
plt.legend()

# Adjust layout and save the DBSCAN + PCA plot
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.savefig('visualizations/DBSCAN_Clusters_PCA.png')
plt.show()


In [ ]:
X = mds_df[mds_cols].copy()

# MDS +DBSCAN Clustering (for MDS-transformed data)
dbscan_best = DBSCAN(eps=best_eps, min_samples=best_min_samples)
best_labels_mds = dbscan_best.fit_predict(X)

# Define a DataFrame with MDS components
mds_df_dbscan = pd.DataFrame(X, columns=['MDS_D1', 'MDS_D2', 'MDS_D3'])
mds_df_dbscan['DBSCAN Clusters'] = best_labels_mds

# Calculate silhouette score for DBSCAN on MDS-transformed data
if len(set(best_labels_mds)) > 1:  # Check if there are more than one cluster
    silhouette_score_mds = silhouette_score(X, best_labels_mds)
else:
    silhouette_score_mds = -1  # Not applicable if there is only one cluster

# Separate plot for DBSCAN + MDS
plt.figure(figsize=(6*3, 6))  # Set figure size
plt.suptitle(f'DBSCAN Clustering of MDS-transformed Data \n(Silhouette score: {round(silhouette_score_mds, 2)})', weight='bold', fontsize=18, y=1.05)

label_list = ['MDS_D1', 'MDS_D2', 'MDS_D3']

# Scatter plot MDS_D1 vs MDS_D2
plt.subplot(1, 3, 1)  # 1 row, 3 columns, 1st subplot
for cluster in mds_df_dbscan['DBSCAN Clusters'].unique():
    cluster_data = mds_df_dbscan[mds_df_dbscan['DBSCAN Clusters'] == cluster]
    label_name = f'Cluster {cluster}' if cluster != -1 else 'Noise'
    plt.scatter(cluster_data['MDS_D1'], cluster_data['MDS_D2'], label=label_name, s=50)
plt.xlabel('MDS_D1')
plt.ylabel('MDS_D2')
plt.legend()

# Scatter plot MDS_D1 vs MDS_D3
plt.subplot(1, 3, 2)  # 1 row, 3 columns, 2nd subplot
for cluster in mds_df_dbscan['DBSCAN Clusters'].unique():
    cluster_data = mds_df_dbscan[mds_df_dbscan['DBSCAN Clusters'] == cluster]
    label_name = f'Cluster {cluster}' if cluster != -1 else 'Noise'
    plt.scatter(cluster_data['MDS_D1'], cluster_data['MDS_D3'], label=label_name, s=50)
plt.xlabel('MDS_D1')
plt.ylabel('MDS_D3')
plt.legend()

# Scatter plot MDS_D2 vs MDS_D3
plt.subplot(1, 3, 3)  # 1 row, 3 columns, 3rd subplot
for cluster in mds_df_dbscan['DBSCAN Clusters'].unique():
    cluster_data = mds_df_dbscan[mds_df_dbscan['DBSCAN Clusters'] == cluster]
    label_name = f'Cluster {cluster}' if cluster != -1 else 'Noise'
    plt.scatter(cluster_data['MDS_D2'], cluster_data['MDS_D3'], label=label_name, s=50)
plt.xlabel('MDS_D2')
plt.ylabel('MDS_D3')
plt.legend()

# Adjust layout and save the MDS plot
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.savefig('visualizations/DBSCAN_Clusters_MDS.png')
plt.show()


- PCA + KMeans: Best Silhoutte Score

In [ ]:
kmeans_df

In [ ]:
kmeans_df.columns

In [ ]:
#select Kmeans PCA
# 
final_unsupervised_df = kmeans_df.copy()
final_unsupervised_df = final_unsupervised_df[['State Name', 'Enplanements', 'Median Household Income',
       'Per Capita Income', 'Gini Index of Income Inequality',
       'Total Population', 'Median Home Value', 'State Code (FIPS)', 'Year',
       'Unemployment Rate', 'Percent Foreigners', 'State Code (USPS)', 'PC1',
       'PC2', 'PC3','KMeans_PCA Clusters']]
final_unsupervised_df.rename(columns={'KMeans_PCA Clusters':'Cluster Number'}, inplace=True)
final_unsupervised_df.head()

In [ ]:
final_unsupervised_df.to_csv('assets/final_unsupervised_df.csv', index=False)

# Supervised Learning

In [3]:
final_unsupervised_df = pd.read_csv('assets/final_unsupervised_df.csv')
final_unsupervised_df.head()

,State Name,Enplanements,Median Household Income,Per Capita Income,Gini Index of Income Inequality,Total Population,Median Home Value,State Code (FIPS),Year,Unemployment Rate,Percent Foreigners,State Code (USPS),PC1,PC2,PC3,Cluster Number
0,Alabama,6.697678e+05,62212.0,35046.0,0.4771,5108468.0,216600.0,1,2023,0.018804,0.039766,AL,-1.767545,1.062621,-0.396554,0
1,Alaska,2.112405e+05,86631.0,45792.0,0.4492,733406.0,347500.0,2,2023,0.023868,0.074408,AK,0.121577,-1.008624,-0.586659,1
2,Arizona,1.227266e+07,77315.0,41290.0,0.4650,7431344.0,411200.0,4,2023,0.020882,0.131999,AZ,0.357048,0.016502,0.373212,1
3,Arkansas,1.096748e+06,58700.0,33012.0,0.4740,3067732.0,195700.0,5,2023,0.019464,0.052919,AR,-2.005391,1.181328,-0.427821,0
4,California,5.954076e+06,95521.0,48013.0,0.4870,38965193.0,725800.0,6,2023,0.028509,0.273065,CA,5.348914,1.898134,2.736928,2


In [4]:
pca_components_df = pd.read_csv('assets/PCA_components.csv')
pca_components_df#.head()

,Median Household Income,Per Capita Income,Gini Index of Income Inequality,Total Population,Median Home Value,Unemployment Rate,Percent Foreigners
0,0.414842,0.414619,0.268479,0.256794,0.409420,0.375605,0.457459
1,-0.430131,-0.304624,0.523909,0.435944,-0.334228,0.364762,0.113597
2,0.045380,-0.348077,-0.461532,0.657799,0.142070,-0.329789,0.319573


- Select three states from each cluster based on three largest population for analysis. (Total population contribute < 50% to PC1 and PC2)

In [5]:
top_states_per_cluster = final_unsupervised_df.groupby('Cluster Number').apply(lambda x: x.nlargest(3, 'Total Population'))
top_states_per_cluster

/var/folders/yc/96hyk6s92717zp_s_6_59khw0000gn/T/ipykernel_1762/3430525690.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top_states_per_cluster = final_unsupervised_df.groupby('Cluster Number').apply(lambda x: x.nlargest(3, 'Total Population'))


State Name  Enplanements  Median Household Income  \
Cluster Number                                                           
0              37  Pennsylvania  2.930786e+06                  73824.0   
               34          Ohio  2.324355e+06                  67769.0   
               9        Georgia  1.174364e+07                  74632.0   
1              12      Illinois  1.004526e+07                  80306.0   
               29    New Jersey  1.729294e+07                  99781.0   
               45      Virginia  1.280296e+06                  89931.0   
2              4     California  5.954076e+06                  95521.0   
               42         Texas  5.727011e+06                  75780.0   
               8        Florida  5.814656e+06                  73311.0   

                   Per Capita Income  Gini Index of Income Inequality  \
Cluster Number                                                          
0              37            42605.0                           0.4774   
               34            39395.0                           0.4695   
               9             39685.0                           0.4787   
1              12            45043.0                           0.4810   
               29            52583.0                           0.4794   
               45            48689.0                           0.4723   
2              4             48013.0                           0.4870   
               42            39775.0                           0.4790   
               8             41902.0                           0.4829   

                   Total Population  Median Home Value  State Code (FIPS)  \
Cluster Number                                                              
0              37        12961683.0           259900.0                 42   
               34        11785935.0           220200.0                 39   
               9         11029227.0           323000.0                 13   
1              12        12549689.0           263300.0                 17   
               29         9290841.0           461000.0                 34   
               45         8715698.0           382900.0                 51   
2              4         38965193.0           725800.0                  6   
               42        30503301.0           296900.0                 48   
               8         22610726.0           381000.0                 12   

                   Year  Unemployment Rate  Percent Foreigners  \
Cluster Number                                                   
0              37  2023           0.021854            0.079567   
               34  2023           0.020652            0.052763   
               9   2023           0.022430            0.115553   
1              12  2023           0.024983            0.149716   
               29  2023           0.025579            0.242220   
               45  2023           0.018889            0.133493   
2              4   2023           0.028509            0.273065   
               42  2023           0.022491            0.178843   
               8   2023           0.020342            0.220996   

                  State Code (USPS)       PC1       PC2       PC3  \
Cluster Number                                                      
0              37                PA -0.012571  1.107356 -0.020235   
               34                OH -0.957689  1.064040  0.129574   
               9                 GA  0.300845  1.095652  0.102730   
1              12                IL  1.184566  1.287278 -0.141721   
               29                NJ  3.351481 -0.120923 -0.051043   
               45                VA  1.053224 -0.524912  0.160013   
2              4                 CA  5.348914  1.898134  2.736928   
               42                TX  1.413576  2.385446  2.112024   
               8                 FL  1.564630  1.720300  1.682971   

                   Cluster Number  
Cluster Number                 

### AIRPORTS

**Largest airport in each top state**
- All of them handle domestic and international flights.

**Cluster 0:**
1. **Pennsylvania: Philadelphia International Airport (PHL)**  - major hub for American Airlines
2. **Ohio: Cleveland Hopkins International Airport (CLE)** 
3. **Georgia: Hartsfield-Jackson Atlanta International Airport (ATL)** - major hub for Delta Air Lines; busiest airport in the world by passenger traffic

**Cluster 1:**
1. **Illinois: O'Hare International Airport (ORD)** - major hub for United Airlines and American Airlines
2. **New Jersey: Newark Liberty International Airport (EWR)** - one of the busiest airports in the United States
3. **Virginia: Washington Dulles International Airport (IAD)** - major hub for United Airlines

**Cluster 2:**
1. **California: Los Angeles International Airport (LAX)** - hub for multiple airlines, including Delta, American, and United
2. **Texas: Dallas/Fort Worth International Airport (DFW)** - major hub for American Airlines
3. **Florida: Miami International Airport (MIA)** - major hub for international travel, especially to Latin America and the Caribbean

### AIRLINES
The selection of airlines will cover:
1. **Full-service (Legacy carriers): Delta/United/American Airlines**, and 
2. **Low-cost carriers: Southwest/Spirit/Frontier Airlines.**

No data:
- IAD_Spirit
- DFW_Southwest

In [120]:
airports = ['PHL','CLE','ATL','ORD','EWR','IAD','LAX','DFW','MIA']
airlines = ['Delta','American','United','Southwest','Spirit','Frontier']

flight_df2 = pd.read_csv('assets/flight_data2.csv')
flight_df2

,Airline Code,Date,Destination Airport Code,Airline,Origin Airport Code,Year,Month,Day
0,DL,2017-01-01,SLC,Delta,PHL,2017,1,1
1,DL,2017-01-01,ATL,Delta,PHL,2017,1,1
2,DL,2017-01-01,ATL,Delta,PHL,2017,1,1
3,DL,2017-01-01,ATL,Delta,PHL,2017,1,1
4,DL,2017-01-01,ATL,Delta,PHL,2017,1,1
...,...,...,...,...,...,...,...,...
6811216,F9,2023-12-31,CLE,Frontier,MIA,2023,12,31
6811217,F9,2023-12-31,MDW,Frontier,MIA,2023,12,31
6811218,F9,2023-12-31,ATL,Frontier,MIA,2023,12,31
6811219,F9,2023-12-31,PHL,Frontier,MIA,2023,12,31


In [121]:
df_airports = pd.read_csv('assets/airport_data.csv')
df_airports.head()

,Airport Code,Airport Name,City,State Code (USPS)
0,ABE,Lehigh Valley International,Allentown,PA
1,ABI,Abilene Regional,Abilene,TX
2,ABQ,Albuquerque International,Albuquerque,NM
3,ACK,Nantucket Memorial,Nantucket,MA
4,ADQ,Kodiak,Kodiak,AK


In [122]:
df_states = pd.read_csv('assets/state_info.csv')
df_states.head()

,State Name,State Code (FIPS),State Code (USPS)
0,Alabama,1,AL
1,Alaska,2,AK
2,Arizona,4,AZ
3,Arkansas,5,AR
4,California,6,CA


In [123]:
df_airports_states = pd.merge(df_airports, df_states, on = 'State Code (USPS)', how = 'left')
df_airports_states

,Airport Code,Airport Name,City,State Code (USPS),State Name,State Code (FIPS)
0,ABE,Lehigh Valley International,Allentown,PA,Pennsylvania,42.0
1,ABI,Abilene Regional,Abilene,TX,Texas,48.0
2,ABQ,Albuquerque International,Albuquerque,NM,New Mexico,35.0
3,ACK,Nantucket Memorial,Nantucket,MA,Massachusetts,25.0
4,ADQ,Kodiak,Kodiak,AK,Alaska,2.0
...,...,...,...,...,...,...
220,TWF,Joslin Field-Magic Valley Regional,Twin Falls,ID,Idaho,16.0
221,TYS,McGhee Tyson,Knoxville,TN,Tennessee,47.0
222,VPS,Eglin Air Force Base,Valparaiso,FL,Florida,12.0
223,WRG,Wrangell,Wrangell,AK,Alaska,2.0


In [115]:
# set(sorted(df_states["State Code (USPS)"].dropna().unique()))^set(sorted(df_airports["State Code (USPS)"].dropna().unique()))
# set(sorted(df_states["State Code (USPS)"].dropna().unique()))
# set(sorted(df_airports["State Code (USPS)"].dropna().unique())) #have starting whitespaces

In [148]:
# Prepare flight data for merging with df_airports_states
flight_df2_origin = flight_df2.drop(columns=[col for col in flight_df2.columns if 'Destination' in col])
flight_df2_origin.rename(columns = {'Origin Airport Code':'Airport Code'}, inplace=True)
flight_df2_origin = flight_df2_origin.merge(df_airports_states, on = 'Airport Code', how = 'left')

In [149]:
# Assign Carrier Type based on the Airline column
flight_df2_origin['Carrier Type'] = [
    'Legacy' if carrier in ['Delta', 'American', 'United'] 
    else 'Low-cost' if carrier in ['Southwest', 'Spirit', 'Frontier'] 
    else 'Other' 
    for carrier in flight_df2_origin['Airline']
]

# Merge main flight data with Cluster information based on best combination (PCA + KMeans) learned in the Unsupervised phase, and
flight_df2_origin = flight_df2_origin.merge(final_unsupervised_df[['State Code (USPS)','Cluster Number']], on='State Code (USPS)', how='left')
flight_df2_origin.head()

,Airline Code,Date,Airline,Airport Code,Year,Month,Day,Airport Name,City,State Code (USPS),State Name,State Code (FIPS),Carrier Type,Cluster Number
0,DL,2017-01-01,Delta,PHL,2017,1,1,Philadelphia International,Philadelphia,PA,Pennsylvania,42.0,Legacy,0
1,DL,2017-01-01,Delta,PHL,2017,1,1,Philadelphia International,Philadelphia,PA,Pennsylvania,42.0,Legacy,0
2,DL,2017-01-01,Delta,PHL,2017,1,1,Philadelphia International,Philadelphia,PA,Pennsylvania,42.0,Legacy,0
3,DL,2017-01-01,Delta,PHL,2017,1,1,Philadelphia International,Philadelphia,PA,Pennsylvania,42.0,Legacy,0
4,DL,2017-01-01,Delta,PHL,2017,1,1,Philadelphia International,Philadelphia,PA,Pennsylvania,42.0,Legacy,0


In [150]:
flight_df2_origin[flight_df2_origin.isna().any(axis=1)] # No rows with missing values

,Airline Code,Date,Airline,Airport Code,Year,Month,Day,Airport Name,City,State Code (USPS),State Name,State Code (FIPS),Carrier Type,Cluster Number


In [151]:
flight_df2_origin.to_csv('assets/flight_data_main.csv', index=False)